<a href="https://colab.research.google.com/github/kenji0011/Chefbot_KasangKap-Hunt/blob/main/Chefbot_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **How to Run the KaSangkap-Hunt (Filipino Chefbot) Project**

This document provides a step-by-step guide to launch and interact with the Filipino Chefbot, a fine-tuned AI model deployed with a Gradio web interface.

# **Prerequisites**
1. A Google Account with access to Google Drive.
2. The project folder, named recipe_project, must be located in the root of "My Drive". This folder should contain:


*   The main Colab notebook file (Chefbot_V3.ipynb)
*   The filipino_recipes.csv dataset.
*   The fully trained model folder: recipe-chef-lora-filipino_v2.
*   The training checkpoints folder: outputs_filipino_v2.



---


# **Step-by-Step Instructions**
The project is designed to be run from a Google Colab notebook. The model is already fully trained, so we only need to run the cells required to load the model and launch the web application.

# **Step 1: Open the Project in Google Colab**
Navigate to Google Drive and open the recipe_project folder.
Double-click the main notebook file (Chefbot_V3.ipynb) to open it in Google Colab.

# **Step 2: Connect to a GPU Runtime**
The model requires a GPU to run effectively.

1. In the top-right corner of the Colab interface, click the
"Connect" button.

2. To ensure you are using a GPU, go to the menu and select "Runtime" -> "Change runtime type".
3. From the "Hardware accelerator" dropdown, select "T4 GPU" and click "Save". The notebook will connect to the new runtime.

# **Step 3: Run the Required Cells to Prepare the Model**
For the Gradio application to function, we must first load all necessary components into the notebook's memory. Please run the following cells in order.

1. Run the Dependencies Installation (**Section 1**)
2. Run the Setup and Data Preparation (**Section 2**)
3. Run the Load Model with Unsloth (**Section 4**)
4. Run the Inference & Model Loading (**Section 9**)

# **Step 4: Launch the Gradio Web Application (Section 10)**
This final step launches the interactive web server.

1. Scroll to the final code cell in the notebook, under the heading Section 10: INTEGRATE WITH GRADIO UI.
2. Run this cell. The code will:
* Define the chatbot's logic.
* Create the chat interface.
* Print a public URL that looks like https://....gradio.live.

# **Step 5: Access and Use the Chefbot**
1. Click on the public gradio.live URL printed in the cell's output.
2. A new browser tab will open, displaying the KaSangkap-Hunt (Filipino Chefbot) interface.
3. You can now interact with the chatbot by typing questions into the input box or clicking on one of the example prompts.

---

**Note**: The Gradio web application will remain live for 72 hours or as long as the Google Colab notebook session is active, whichever is shorter. If the session is disconnected, you will need to repeat Steps 2 through 5 to generate a new public URL.

# **1. REQUIRED DEPENDENCIES - Skip if not disconnected session**

In [ ]:
# Install Unsloth and other required packages.
# This is optimized for Google Colab with a T4 GPU.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes gradio

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-l4a15m2p/unsloth_6afaef2d912c41e8982cac835f73ef97
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-l4a15m2p/unsloth_6afaef2d912c41e8982cac835f73ef97
  Resolved https://github.com/unslothai/unsloth.git to commit 8490f6efc407f409c42081988e93973df8e11f2d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 144.6 MB/s eta 0:00:00

# **2. SETUP AND DATA PREPARATION**

In [ ]:

import pandas as pd
import random
import os
from google.colab import drive
from datasets import Dataset

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- Define File Paths ---
DRIVE_PROJECT_PATH = "/content/drive/MyDrive/recipe_project"
DATASET_PATH = os.path.join(DRIVE_PROJECT_PATH, "filipino_recipes.csv")

# Using new v4 folders for the next training run
OUTPUT_DIR = os.path.join(DRIVE_PROJECT_PATH, "outputs_filipino_v4")
LORA_MODEL_PATH = os.path.join(DRIVE_PROJECT_PATH, "recipe-chef-lora-filipino_v4")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LORA_MODEL_PATH, exist_ok=True)


# --- Load and Prepare Dataset ---
print("Loading and preparing dataset from filipino_recipes.csv...")
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"❌ Dataset not found at {DATASET_PATH}.")

df = pd.read_csv(DATASET_PATH)
df.dropna(subset=['recipe_name', 'ingredients', 'instructions'], inplace=True)

training_data = []
# Using df.itertuples() for a slight performance increase
for index, row in enumerate(df.itertuples(index=False)):
    title = row.recipe_name.strip()
    ingredients_str = row.ingredients
    instructions_str = row.instructions
    total_time = getattr(row, 'total_time', None)

    try:
        ingredients = [ing.strip() for ing in ingredients_str.split('|')]
        steps = [step.strip() for step in instructions_str.split('|')]
    except AttributeError:
        continue

    # --- Standard Recipe Generation ---
    recipe_text = "Ingredients:\n" + "\n".join(f"- {ing}" for ing in ingredients)
    recipe_text += "\n\nInstructions:\n" + "\n".join(f"{i}. {step}" for i, step in enumerate(steps, 1))
    prompts = [f"How do you make {title}?", f"Write a full recipe for {title}."]
    for prompt in prompts:
        training_data.append({"instruction": prompt, "input": "", "output": recipe_text.strip()})

    # --- Enhanced Substitution Logic ---
    if ingredients:
        substitutions = {
            'pork': 'chicken for a lighter adobo, or beef for a richer flavor.',
            'pork belly': 'oxtail (traditional for Kare-Kare), beef chuck, or even chicken pieces.',
            'beef': 'pork or chicken. For stews, a tougher cut of pork like shoulder is a good choice.',
            'chicken': 'pork for a richer adobo, or firm tofu for a vegetarian option.',
            'shrimp': 'scallops, fish fillet, or squid rings. Adjust cooking time as needed.',
            'fish sauce': 'soy sauce with a squeeze of calamansi for a similar salty-sour profile.',
            'bagoong': 'fish sauce (patis) can provide the saltiness, but the unique fermented flavor will be missing.',
            'vinegar': 'calamansi juice or lemon juice can provide acidity, but will change the flavor profile.',
            'soy sauce': 'fish sauce for saltiness, or coconut aminos for a gluten-free option.'
        }
        generated_count = 0
        for ingredient_to_replace in ingredients:
            if generated_count >= 2: break
            for key, value in substitutions.items():
                if key in ingredient_to_replace.lower():
                    training_data.append({
                        "instruction": f"What can I use instead of {ingredient_to_replace} in {title}?",
                        "input": "",
                        "output": f"For {title}, a common substitute for {ingredient_to_replace} is {value}. Make sure to adjust cooking times if necessary."
                    })
                    generated_count += 1
                    break
        if generated_count == 0:
             training_data.append({
                "instruction": f"Can I substitute {random.choice(ingredients)} in {title}?",
                "input": "",
                "output": f"For many ingredients in {title}, you can experiment with substitutes that have a similar taste and texture. It's a great way to be creative in the kitchen!"
            })

    # --- Time-based Question ---
    if total_time and pd.notna(total_time):
         training_data.append({
            "instruction": f"How long does it take to make {title}?",
            "input": "",
            "output": f"It takes about {total_time} to prepare and cook {title}."
        })

    # NEWLY ADDED PERSONA TRAINING DATA
    # --- Style and Persona-Based Generation ---
    # This teaches the model how to adopt different personas.
    # It creates an explanation for a "beginner" for roughly 1 out of every 20 recipes.
    if index % 20 == 0 and steps:
        beginner_steps = []
        for i, step in enumerate(steps, 1):
            # Add helpful, encouraging phrases to the instructions
            if i == 1:
                beginner_steps.append(f"{i}. Don't worry, this is the easy part! First, {step[0].lower() + step[1:]}")
            elif i == len(steps):
                beginner_steps.append(f"{i}. You're almost there! Finally, {step[0].lower() + step[1:]}")
            else:
                beginner_steps.append(f"{i}. Great job! Now, let's {step[0].lower() + step[1:]}")

        beginner_instructions = "\n".join(beginner_steps)
        beginner_output = (
            f"Of course! Making {title} is fun and I'd be happy to guide you. Here’s a simple guide perfect for a beginner:\n\n"
            f"**Ingredients:**\n" + "\n".join(f"- {ing}" for ing in ingredients) +
            f"\n\n**Instructions:**\n" + beginner_instructions +
            f"\n\n**Beginner's Tip:** Don't be afraid to taste as you go! Cooking is all about adjusting flavors to your liking. Enjoy the process!"
        )

        training_data.append({
            "instruction": f"Explain how to make {title} as if you were talking to a complete beginner at cooking.",
            "input": "",
            "output": beginner_output
        })

print(f"✅ Created {len(training_data)} training examples.")

Mounted at /content/drive
Loading and preparing dataset from filipino_recipes.csv...
✅ Created 9282 training examples.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **3. Convert to Hugging Face Dataset**

In [ ]:
dataset = Dataset.from_list(training_data)
print("\nSample training data point:")
print(dataset[0])


Sample training data point:
{'instruction': 'How do you make Pork Hock and Mung Beans with Vermicelli (Ginisang Monggo with Pata at Sotanghon)?', 'input': '', 'output': 'Ingredients:\n- 3 lbs. pork hocks cleaned\n- 1 1/2 cups mung beans washed\n- 1 ounce sotanghon\n- 2 teaspoons beef powder\n- 1 onion chopped\n- 5 garlic chopped\n- 3 tomatoes diced\n- 2 cups baby spinach\n- 8 cups water\n- Fish sauce and ground black pepper to taste\n- 3 tablespoons cooking oil\n\nInstructions:\n1. Boil the pork hock in 4 cups of water for 10 minutes. Drain the water and then wash the pork to ensure that it is clean. Set aside.\n2. Heat oil in a cooking pot. Saute garlic, onion and tomatoes.\n3. Once the onion and tomato soften, add the mung beans. Saut for 30 seconds.\n4. Pour 4 cups of water into the pot. Add the pork hock. Let boil. Cover and simmer for 50 to 60 minutes or until the pork tenderizes. Occasionally stir every 10 minutes and add more water as necessary.\n5. Add the sotanghon noodles. S

# **4. LOAD MODEL WITH UNSLOTH**

In [ ]:
import torch
from unsloth import FastLanguageModel

# Configuration
max_seq_length = 2048
dtype = None  # Autodetect
load_in_4bit = True  # Use 4-bit quantization for memory efficiency

print("\nLoading base model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("✅ Base model loaded.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.



Loading base model...
==((====))==  Unsloth 2025.12.5: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Base model loaded.


# **5. CONFIGURE LoRA (PEFT)**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)
print("✅ LoRA configuration applied.")

Unsloth 2025.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ LoRA configuration applied.


# **6. FORMAT DATASET FOR TRAINING**

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="mistral",
)

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Apply the chat template for instruction-following
        text = tokenizer.apply_chat_template(
            [{"role": "user", "content": instruction}, {"role": "assistant", "content": output}],
            tokenize=False,
            add_generation_prompt=False,
        )
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/9282 [00:00<?, ? examples/s]

# **7. TRAIN THE MODEL**

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.trainer_utils import get_last_checkpoint

# Split dataset into train and evaluation sets
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=2400,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=200,   # Let's make this less noisy
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir=OUTPUT_DIR,
        save_strategy="steps",
        save_steps=200,
        save_total_limit=3,
        eval_steps=200,
        report_to="tensorboard",
    ),
)


# Check if a checkpoint exists in the output directory on your Google Drive
last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

print("\nStarting training...")
# This will now resume from 'last_checkpoint' if it exists,
# or start a new training run if the folder is empty.
trainer_stats = trainer.train(resume_from_checkpoint=last_checkpoint)

print("✅ Training finished.")

Map (num_proc=2):   0%|          | 0/8353 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/929 [00:00<?, ? examples/s]


Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,353 | Num Epochs = 3 | Total steps = 2,400
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
2200,0.631000
2400,0.636500


✅ Training finished.


# **8. SAVE THE FINAL ADAPTER**

In [ ]:
print("\nSaving final LoRA adapter to Google Drive...")
model.save_pretrained(LORA_MODEL_PATH)
tokenizer.save_pretrained(LORA_MODEL_PATH)
print(f"✅ Model saved to {LORA_MODEL_PATH}")


Saving final LoRA adapter to Google Drive...
✅ Model saved to /content/drive/MyDrive/recipe_project/recipe-chef-lora-filipino_v4


# **9. INFERENCE (LOAD MODEL FROM DRIVE AND TEST)**

In [ ]:
# This section shows how to load the trained model for inference in a new session.
# You MUST run sections 1, 2 (for paths), and 4 (to get the base model) before this.

from peft import PeftModel

# --- Load the base model first ---
print("\n--- INFERENCE MODE ---")
print("Loading base model for inference...")
# You must load the same base model you used for training
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# --- Apply your trained LoRA adapter ---
print(f"Loading LoRA adapter from: {LORA_MODEL_PATH}")
model = PeftModel.from_pretrained(model, LORA_MODEL_PATH)
print("✅ LoRA adapter loaded successfully.")

# --- Prepare model for faster inference ---
FastLanguageModel.for_inference(model)

# --- Inference Function ---
def generate_recipe(prompt):
    messages = [{"role": "user", "content": prompt}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        use_cache=True,
        repetition_penalty=1.15, # Discourages the model from repeating itself.
        temperature=0.6,         # Makes the output a bit more focused.
        top_p=0.9,
        do_sample=True,
    )
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return response.split("[/INST]")[-1].strip()

# --- Test Inference ---
print("\n--- Testing Model Generation ---")
print(generate_recipe("Write a recipe for Chicken Adobo."))
print("\n" + "="*50 + "\n")
print(generate_recipe("How long does it take to make Sinigang Sa Miso?"))


--- INFERENCE MODE ---
Loading base model for inference...
==((====))==  Unsloth 2025.12.5: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Loading LoRA adapter from: /content/drive/MyDrive/recipe_project/recipe-chef-lora-filipino_v4


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


✅ LoRA adapter loaded successfully.

--- Testing Model Generation ---
Write a recipe for Chicken Adobo.Ingredients:
- 1 kilogram chicken (cut into serving pieces)
- 4 tablespoons soy sauce
- 3 cloves Garlic (crushed)
- 1 piece bay leaf
- 2 teaspoons peppercorns
- 1/4 Cup water
- 5 to 6 tablespoons Vinegar
- Salt (to taste)

Instructions:
1. In a bowl, mix together the chicken and all other ingredients except salt. Let it marinate overnight in the refrigerator.
2. Transfer the mixture in a pot and simmer over low heat until the chicken is cooked through and tender.
3. Season with salt according to your taste preference. Serve warm or at room temperature.


How long does it take to make Sinigang Sa Miso?It takes about 1 hour to prepare and cook Sinigang Sa Miso.


# **10. RUN WITH GRADIO UI (WORKS, FASTEST)**

In [ ]:
import gradio as gr
import torch

# --- Chatbot Function with Streaming Output ---
def generate_recipe_chat(message, history):
    messages = []
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": message})

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    input_length = inputs.shape[1]
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=512,
            use_cache=True,
            repetition_penalty=1.15,
            temperature=0.6
        )

    new_tokens = outputs[:, input_length:]
    response = tokenizer.batch_decode(new_tokens, skip_special_tokens=True)[0]

    full_response = ""
    for char in response.strip():
        full_response += char
        yield full_response

# --- Create a Lively Custom Theme (Simplified) ---
lively_theme = gr.themes.Soft(
    primary_hue=gr.themes.colors.teal,
    secondary_hue=gr.themes.colors.orange,
    neutral_hue=gr.themes.colors.slate,
    font=[gr.themes.GoogleFont("Poppins"), "sans-serif"],
).set(
    body_background_fill="#F0F2F6", # Light grey background
    block_background_fill="white",
    block_border_width="1px",
    button_primary_text_color="white",
)

# --- Create and Launch the Gradio Chat Interface ---
demo = gr.ChatInterface(
    fn=generate_recipe_chat,
    title="🧑‍🍳 KaSangkap-Hunt (Filipino Chefbot)",
    description="I can generate Filipino recipes, answer questions, and more! Ask me anything.",
    examples=[
        "Write a recipe for Pork Sinigang. Provide the instructions as a step-by-step numbered list.",
        "How do you cook Lechon Kawali?",
        "What can I use instead of pork in Adobo?",
    ],
    chatbot=gr.Chatbot(height=700),
    textbox=gr.Textbox(placeholder="Ask me about a Filipino recipe...", container=False, scale=7),
    theme=lively_theme
)

# Launch the app with a public URL
demo.launch(share=True)

/tmp/ipython-input-1584642283.py:60: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=700),
/tmp/ipython-input-1584642283.py:60: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot=gr.Chatbot(height=700),
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff687a0436006d62d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **LAUNCH THE STREAMLIT WEB APP (NEED AUTHTOKEN, SLOW)**

In [ ]:
"""# 1. Install ALL necessary libraries for the app to run in a fresh session.
# This includes unsloth, its dependencies, Streamlit, and pyngrok.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install streamlit pyngrok

# 2. Import the tools we need for launching.
from pyngrok import ngrok
import os
import getpass

# 3. Set the working directory to your web_app folder.
# This is crucial so the script can find 'streamlit_app.py'.
os.chdir('/content/drive/MyDrive/recipe_project/web_app')

# 4. Terminate any old ngrok tunnels to prevent errors.
ngrok.kill()

# 5. Get your ngrok token and set it up.
# Get a free token from https://dashboard.ngrok.com/get-started/your-authtoken
print("Please paste your ngrok authtoken, then press Enter:")
NGROK_AUTH_TOKEN = getpass.getpass()
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 6. Run the Streamlit app in the background.
# The '&>/dev/null&' part is important to allow the rest of the cell to run.
!streamlit run streamlit_app.py &>/dev/null&

# 7. Create the public URL and print it.
# Streamlit's default port is 8501.
public_url = ngrok.connect(8501)
print("====================================================================")
print(f"✅ Your Filipino Chefbot is LIVE at: {public_url}")
print("--- Please wait 5-10 minutes for the model to load before visiting the URL. ---")
print("====================================================================")

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-p2gd9nok/unsloth_a82ac7d919ac426cb910f9bb5508dc28
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-p2gd9nok/unsloth_a82ac7d919ac426cb910f9bb5508dc28
  Resolved https://github.com/unslothai/unsloth.git to commit d4a311d8e71692961e5da1d26d98197fde94f41a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.24.0
    Uninstalling trl-0.24.0:
      Successfully 